In [1]:
import pandas as pd

In [2]:
data_url = "https://raw.githubusercontent.com/fenago/tf/main/Chapter6-Regularization_and_Hyperparameter_Tuning/dataset/shuttle.trn"

data = pd.read_table(data_url, header=None, sep=' ')
data.head()

,0,1,2,3,4,5,6,7,8,9
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1
4,37,0,79,0,34,-26,43,46,2,1


In [3]:
y = data.pop(9)
X = data.copy()

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)

In [8]:
!pip install keras-tuner
import keras_tuner as kt

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [10]:
tf.random.set_seed(8)

In [22]:
def model_builder(hp):
        model = tf.keras.Sequential()
        hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001, 0.0001])
        reg_fc1 = Dense(512, input_shape=(9,), activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc2 = Dense(512, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc3 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc4 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc5 = Dense(8, activation='softmax')

        model.add(reg_fc1)
        model.add(reg_fc2)
        model.add(reg_fc3)
        model.add(reg_fc4)
        model.add(reg_fc5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(0.001)
        model.compile(optimizer = optimizer, loss = loss, \
                      metrics = ['accuracy'])
        return model

In [23]:
tuner = kt.RandomSearch(model_builder, objective='val_accuracy', \
                            max_trials=10)

In [24]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.9576244950294495

Best val_accuracy So Far: 0.9956321716308594
Total elapsed time: 00h 00m 15s
INFO:tensorflow:Oracle triggered exit


In [25]:
best_hps = tuner.get_best_hyperparameters()[0]

In [26]:
best_l2 = best_hps.get('l2')
best_l2

0.0001

In [27]:
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=5, \
              validation_data=(X_test, y_test))

Epoch 1/5
952/952 [==============================] - 3s 3ms/step - loss: 0.4201 - accuracy: 0.9711 - val_loss: 0.2969 - val_accuracy: 0.9810
Epoch 2/5
952/952 [==============================] - 3s 3ms/step - loss: 0.1842 - accuracy: 0.9926 - val_loss: 0.1526 - val_accuracy: 0.9965
Epoch 3/5
952/952 [==============================] - 3s 3ms/step - loss: 0.1360 - accuracy: 0.9947 - val_loss: 0.2095 - val_accuracy: 0.9939
Epoch 4/5
952/952 [==============================] - 3s 3ms/step - loss: 0.1191 - accuracy: 0.9941 - val_loss: 0.1629 - val_accuracy: 0.9848
Epoch 5/5
952/952 [==============================] - 3s 3ms/step - loss: 0.0819 - accuracy: 0.9966 - val_loss: 0.0898 - val_accuracy: 0.9967
